In [1]:
# regular data science imports
import pandas as pd 
import numpy as np 

# to handle conversion of string to vectors
import ast

# to handle regular expression matching
import re

# to handle file paths / directories
import os

# to build the Bayesian Network 
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import BayesianEstimator
from pgmpy.inference import VariableElimination

In [2]:
# visualize the data the compact data
# this file has the feature vectors but are listed as strings
root_dir = os.getcwd()
data_path = os.path.join(root_dir, "..", "dichotmous-keys", "compact_descriptions.csv")
df = pd.read_csv(data_path, encoding="latin1")
df.head()


,Species,Full description,Full vector
0,garnhami,Abdominal segments not with laterally projecti...,"""[0, 0, 0, 0, 0, 1, 0, 1, nan, nan, nan, nan, ..."
1,brumpti,Abdominal segments with laterally projecting t...,"""[1, nan, nan, nan, nan, nan, nan, nan, nan, n..."
2,stephensi,Abdominal segments not with laterally projecti...,"""[0, 0, 0, 1, nan, nan, nan, nan, nan, nan, na..."
3,marshallii,Abdominal segments not with laterally projecti...,"""[0, 0, 0, 0, 0, 1, 0, 0, 1, nan, nan, nan, na..."
4,NaN,Abdominal segments not with laterally projecti...,"""[0, 1, nan, 0, nan, nan, nan, nan, nan, nan, ..."


In [3]:
# we need to get the feature names as well
feature_path = os.path.join(root_dir, "..", "dichotmous-keys", "text_base.csv")
df_features = pd.read_csv(feature_path, encoding="latin1")
df_features.head()

,Unnamed: 0,Unnamed: 1,Section 0,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Section XI,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130
0,NaN,NaN,1,2,3,4,5,6,7,8,...,120,121,122,123,124,125,126,127,128,129
1,B statements (true when cell value is 0):,NaN,Abdominal segments not with laterally projecti...,Hindtarsus with at most of last 2 hindtarsomer...,"Hindtarsomere 5 not mainly or entirely dark, o...",Legs not speckled,"Wing not entirely dark, with pale spots not co...",Wing without a pale spot on basal 0.5 of costa,Maxillary palpus with apex pale,Maxillary palpus with less than 4 pale bands,...,Maxillary palpus with 3 pale bands,"Base of costa dark or with small pale spot, ba...",These veins dark,"Subapical pale spot shorter, usually much shor...",Pale banding on hindtarsomeres narrow and apic...,"Preaccessory dark spot absent or, if present, ...",Basal area of wing vein 1 entirely pale,Subapical pale band on maxillary palpus much s...,"Small species, wing length 3.2 mm or less","Tip of wing vein 6 with a few pale scales, som..."
2,Species,Region,Abdominal segments with laterally projecting t...,Hindtarsus with at least last 2 hindtarsomeres...,"Hindtarsomere 5 mainly or entirely dark, hindt...","Legs speckled, sometimes sparsely",Wing entirely dark or with pale spots confined...,Wing with at least 1 pale spot on basal 0.5 of...,Maxillary palpus with apex dark,Maxillary palpus with 4 pale bands,...,Maxillary palpus with only apex pale,Base of costa with large (presector) pale spot...,Lower branch of wing vein 2 and upper branch o...,Subapical pale spot on costa and wing vein 1 a...,Hindtarsomeres 1 to 4 with pale bands overlapp...,Preaccessory dark spot on wing vein 1 about tw...,Basal area of wing vein 1 proximal to 1st main...,Subapical pale band on maxillary palpus longer...,"Moderate-sized species, wing length more than ...",Tip of wing vein 6 dark with no fringe spot
3,pharoensis,NaN,Abdominal segments with laterally projecting t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,coustani,NaN,Abdominal segments not with laterally projecti...,Hindtarsus with at least last 2 hindtarsomeres...,NaN,Legs not speckled,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# get feature vectors
feature_vectors = df["Full vector"].apply(
    lambda x: np.array(
        ast.literal_eval(
            ast.literal_eval(
                x.replace('nan', '0')      # if it's nan, just give it a 0
            )
        )
    )
)

print(feature_vectors)
print('Type: ', type(feature_vectors[0]))   # confirm that our conversion worked

0      [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...
1      [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...
2      [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3      [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
4      [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                             ...                        
112    [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...
113    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
114    [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...
115    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
116    [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: Full vector, Length: 117, dtype: object
Type:  <class 'numpy.ndarray'>


In [5]:
# get the feature names from text-base
tmp = df_features.iloc[2]
tmp = tmp[2:]
feature_names = tmp.tolist()

# some of these have question marks at the end... gotta remove those
print(feature_names)
print('Number of features ', len(feature_names))

['Abdominal segments with laterally projecting tufts of scales on segments II-VII?', 'Hindtarsus with at least last 2 hindtarsomeres entirely pale?', 'Hindtarsomere 5 mainly or entirely dark, hindtarsomere 4 white', 'Legs speckled, sometimes sparsely', 'Wing entirely dark or with pale spots confined to costa and vein 1', 'Wing with at least 1 pale spot on basal 0.5 of costa', 'Maxillary palpus with apex dark', 'Maxillary palpus with 4 pale bands', 'Wing with pale interruption in 3rd main dark area (preapical dark spot) of vein 1, sometimes fused with preceding pale area', 'Wing with 2 pale spots on upper branch of vein 5', 'Wing almost entirely dark, costa without pale spots', 'Hindtarsomeres 1 to 5 entirely dark', 'Hindtarsomeres 1 and 2 with definite pale and dark rings in addition to apical pale bands', 'Hindtarsomeres 3 and 4 all white or narrowly dark basally, 5 all dark or at least basal 0.5 dark', 'Moderate-sized species; abdominal scale-tufts short and dark; 0.5 or more of hind

In [ ]:
# optional... make the names easier to process 
# clean_names = [re.sub(r'[^A-Za-z0-9_]+', '_', f).strip('_') for f in feature_names]
clean_names = [f.replace("?", "") for f in feature_names]

['Abdominal segments with laterally projecting tufts of scales on segments II-VII', 'Hindtarsus with at least last 2 hindtarsomeres entirely pale', 'Hindtarsomere 5 mainly or entirely dark, hindtarsomere 4 white', 'Legs speckled, sometimes sparsely', 'Wing entirely dark or with pale spots confined to costa and vein 1', 'Wing with at least 1 pale spot on basal 0.5 of costa', 'Maxillary palpus with apex dark', 'Maxillary palpus with 4 pale bands', 'Wing with pale interruption in 3rd main dark area (preapical dark spot) of vein 1, sometimes fused with preceding pale area', 'Wing with 2 pale spots on upper branch of vein 5', 'Wing almost entirely dark, costa without pale spots', 'Hindtarsomeres 1 to 5 entirely dark', 'Hindtarsomeres 1 and 2 with definite pale and dark rings in addition to apical pale bands', 'Hindtarsomeres 3 and 4 all white or narrowly dark basally, 5 all dark or at least basal 0.5 dark', 'Moderate-sized species; abdominal scale-tufts short and dark; 0.5 or more of hindta

In [28]:
# save the possible feature names as a json
import json
with open("feature_names.json", "w") as f:
    json.dump(clean_names, f, indent=2)

In [20]:
# create a data frame where column 1 = species name and then the subsequent columns are feature names
dat = pd.DataFrame(df["Species"], columns=["Species"])

# get rid of invalid species names
dat = dat.dropna(subset=["Species"]) 
tmp = pd.DataFrame(0, index=dat.index, columns=clean_names)
dat = pd.concat([dat, tmp], axis=1)

N = len(dat)
for i in range(len(dat)):
    dat.iloc[i, 1:] = feature_vectors[i]
    
dat.fillna(0, inplace=True)  
dat.head()

,Species,Abdominal segments with laterally projecting tufts of scales on segments II-VII,Hindtarsus with at least last 2 hindtarsomeres entirely pale,"Hindtarsomere 5 mainly or entirely dark, hindtarsomere 4 white","Legs speckled, sometimes sparsely",Wing entirely dark or with pale spots confined to costa and vein 1,Wing with at least 1 pale spot on basal 0.5 of costa,Maxillary palpus with apex dark,Maxillary palpus with 4 pale bands,"Wing with pale interruption in 3rd main dark area (preapical dark spot) of vein 1, sometimes fused with preceding pale area",...,Maxillary palpus with only apex pale,"Base of costa with large (presector) pale spot, base of vein 1 pale",Lower branch of wing vein 2 and upper branch of vein 4 with distinct pale spots,"Subapical pale spot on costa and wing vein 1 about as long as apical dark spot, fringe spots present opposite veins 3, lower branch of 4 and both branches of 5","Hindtarsomeres 1 to 4 with pale bands overlapping the joints, at least hindtarsomere 5 pale basally",Preaccessory dark spot on wing vein 1 about twice as long as pale spot on either side of it,"Basal area of wing vein 1 proximal to 1st main dark area, pale with a broad dark spot",Subapical pale band on maxillary palpus longer than or equal to apical dark band AND 3rd main dark area of costa and vein 1 equal to or shorter than subapical pale spot,"Moderate-sized species, wing length more than 3.3 mm",Tip of wing vein 6 dark with no fringe spot
0,garnhami,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,brumpti,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,stephensi,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,marshallii,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,namibiensis,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# species check
print("gambiae" in dat["Species"].values)
print("funestus" in dat["Species"].values)
print("coustani" in dat["Species"].values) # uganda

print('columbia:')
print("albimanus" in dat["Species"].values)  # colombia
print("nuneztovari" in dat["Species"].values) # colombia

print('mozambique:')
print("pharoensis" in dat["Species"].values) # mozambique
print("tenebrosus" in dat["Species"].values) # mozambique

True
True
True
columbia:
False
False
mozambique:
True
True


In [22]:
# create basic BN model

# extract the which features are present in each species of mosquito
features = [col for col in dat.columns if col != "Species"]

# build the model
nodes = [("Species", f) for f in features]
model = DiscreteBayesianNetwork(nodes)

# for some reason it's intepretting types weird so chatgpt suggested type casting
# the species name to a string...which it should already be one.. 
dat["Species"] = dat["Species"].astype(str)

# Fit with a Dirichlet Prior to keep probabilties smooth
model.fit(
    dat,     # our data frame
    estimator=BayesianEstimator,
    prior_type="BDeu",
    equivalent_sample_size=5
)

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'Species': 'C', 'Abdominal segments with laterally projecting tufts of scales on segments II-VII': 'N', 'Hindtarsus with at least last 2 hindtarsomeres entirely pale': 'N', 'Hindtarsomere 5 mainly or entirely dark, hindtarsomere 4 white': 'N', 'Legs speckled, sometimes sparsely': 'N', 'Wing entirely dark or with pale spots confined to costa and vein 1': 'N', 'Wing with at least 1 pale spot on basal 0.5 of costa': 'N', 'Maxillary palpus with apex dark': 'N', 'Maxillary palpus with 4 pale bands': 'N', 'Wing with pale interruption in 3rd main dark area (preapical dark spot) of vein 1, sometimes fused with preceding pale area': 'N', 'Wing with 2 pale spots on upper branch of vein 5': 'N', 'Wing almost entirely dark, costa without pale spots': 'N', 'Hindtarsomeres 1 to 5 entirely dark': 'N', 'Hindtarsomeres 1 and 2 with definite pale and dark rings in addition to apical pale bands': 'N'

In [25]:
# save the network
import joblib

joblib.dump(model, "BN.pkl")

['BN.pkl']

### Usage of the Network Model

In [23]:
# show probability tables for each species based on the presence or absence of a feature

# cpd = conditional probability distribution
def cpd_to_df(cpd):
    species = list(cpd.state_names["Species"])
    df = pd.DataFrame(cpd.values, 
                      index=cpd.state_names[cpd.variable],
                      columns=species)
    
    # if there are na values I missed, this will give an error
    if len(df.index) == 2:
        df.index = ["No", "Yes"]
        
    return df

all_cpds = {
    cpd.variable: cpd_to_df(cpd)
    for cpd in model.get_cpds()
    if cpd.variable != "Species"
}

all_cpds["Femora and tibiae speckled"].head()



,ardensis,argenteolobatus,aruni,austenii,azaniae,azevedoi,barberellus,berghei,bervoetsi,brohieri,...,tchekedii,tenebrosus,turkhudi,vanhoofi,vernus,vinckei,walravensi,wellcomei,wilsoni,ziemanni
No,0.979167,0.979167,0.979167,0.979167,0.979167,0.979167,0.979167,0.979167,0.979167,0.979167,...,0.979167,0.979167,0.979167,0.979167,0.979167,0.979167,0.979167,0.979167,0.979167,0.979167
Yes,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,...,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833,0.020833


In [24]:
# can also try to do some inference using the model by faking some example input
inference = VariableElimination(model)

query_result = inference.query(
    variables=["Species"],   # what you want to predict
    evidence={
        "Wing with at least 1 pale spot on basal 0.5 of costa": 1,
        "Femora and tibiae speckled": 0,
        "Maxillary palpus with 4 pale bands": 1,
        "Maxillary palpus with apex dark": 0
    }
)

# Extract species names and probabilities
species_names = query_result.state_names["Species"]
probs = query_result.values

# Find index of the max probability
max_idx = probs.argmax()

# Print result (low probability because we didn't describe a lot of distinguishing features)
print("Most likely species:", species_names[max_idx])
print("Probability:", probs[max_idx])

Most likely species: brohieri
Probability: 0.15669273544884946


In [18]:
# to print the sorted list of predicitons we can write:

species = np.array(query_result.state_names["Species"])
probs = np.array(query_result.values)

top10_idx = np.argsort(probs)[::-1][:10]  # sort descending, take top 10
top10_species = species[top10_idx]
top10_probs = probs[top10_idx]

for sp, p in zip(top10_species, top10_probs):
    print(f"{sp:25s}  P={p:.4f}")

machardyi                  P=0.1567
garnhami                   P=0.1567
longipalpis                P=0.1567
brohieri                   P=0.1567
dthali                     P=0.1567
caroni                     P=0.0817
tchekedii                  P=0.0033
vinckei                    P=0.0033
turkhudi                   P=0.0033
seretsei                   P=0.0033
